In [ ]:
!pip install -q transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
# 1.Load and inspect the dataset
# Load the emotion classification dataset from Hugging Face
dataset = load_dataset("emotion")

# Print dataset sizes to understand the data split
print(f"Train samples: {len(dataset['train'])}")
print(f"Test samples: {len(dataset['test'])}")
print(f"Validation samples: {len(dataset['validation'])}")

# Inspect a single training sample
print("\nSample:")
print(dataset['train'][0])

# Extract label names for later use
label_names = dataset['train'].features['label'].names
print(f"\nLabels: {label_names}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Train samples: 16000
Test samples: 2000
Validation samples: 2000

Sample:
{'text': 'i didnt feel humiliated', 'label': 0}

Labels: ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [ ]:
# 2.Model and tokenizer selection
# Use a lightweight BERT based model suitable for text classification
model_name = "distilbert-base-uncased"
num_labels = len(dataset['train'].features['label'].names)


# Load tokenizer and model with correct number of output labels
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 3.Text tokenization
# Tokenization function converts raw text into model input format
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization to the entire dataset
# Remove raw text column since the model only needs tokenized inputs
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# 4.Training configuration
# Define training arguments for fine tuning
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    load_best_model_at_end=True,
)

# 5.Evaluation metrics
# Custom metric function for accuracy and F1 score
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

In [ ]:
# 6.Trainer initialization
# Hugging Face Trainer handles training and evaluation loop
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

# 7.Model training
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.264600,0.217328,0.919500,0.920349
2,0.156100,0.145369,0.941000,0.941114
3,0.124200,0.138955,0.942000,0.942124


TrainOutput(global_step=1500, training_loss=0.2973488076527913, metrics={'train_runtime': 571.2373, 'train_samples_per_second': 84.028, 'train_steps_per_second': 2.626, 'total_flos': 1589722177536000.0, 'train_loss': 0.2973488076527913, 'epoch': 3.0})

In [ ]:
# 8.Final evaluation on test set
results = trainer.evaluate(tokenized_datasets['test'])
print(f"Accuracy: {results['eval_accuracy']:.4f}")
print(f"F1 Score: {results['eval_f1']:.4f}")

# Generate detailed classification report
predictions = trainer.predict(tokenized_datasets['test'])
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = tokenized_datasets['test']['label']

print("\n" + classification_report(true_labels, pred_labels, target_names=label_names))

Accuracy: 0.9220
F1 Score: 0.9218

              precision    recall  f1-score   support

     sadness       0.96      0.96      0.96       581
         joy       0.95      0.94      0.94       695
        love       0.81      0.84      0.82       159
       anger       0.93      0.92      0.92       275
        fear       0.88      0.91      0.89       224
    surprise       0.70      0.64      0.67        66

    accuracy                           0.92      2000
   macro avg       0.87      0.87      0.87      2000
weighted avg       0.92      0.92      0.92      2000



In [ ]:
# 9.Inference function
# Function for predicting emotion from raw text
def predict_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Move inputs to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    # Disable gradient calculation for inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract predicted class and confidence
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    confidence = torch.softmax(outputs.logits, dim=1)[0][predicted_class].item()
    return label_names[predicted_class], confidence

In [ ]:
# Example predictions
test_texts = [
    "I am so happy today!",
    "This is terrible and disappointing",
    "I'm scared of what might happen",
    "I love this so much",
]

for text in test_texts:
    emotion, conf = predict_emotion(text)
    print(f"Text: {text}")
    print(f"Emotion: {emotion} (confidence: {conf:.2%})\n")

Text: I am so happy today!
Emotion: joy (confidence: 99.15%)

Text: This is terrible and disappointing
Emotion: sadness (confidence: 99.12%)

Text: I'm scared of what might happen
Emotion: fear (confidence: 99.04%)

Text: I love this so much
Emotion: love (confidence: 53.99%)



In [ ]:
# Save model and tokenizer for later use or deployment
trainer.save_model("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')